In [ ]:
# BerkAI 03 & 04 Intro & Funda of Data Analysis

In [ ]:
The CRISP-DM Framework

Understand this in conjunction with PMP learnings of Data in ITTOs (Data gathering, Data analysis, Data visualization)

In [ ]:
# Joining Datasets

In [1]:
# Import Libs

# lib for files access
from pathlib import Path
myfolder = Path("C:/Users/pdpc/Documents/MLfiles/Data/zsrv_sarirasri/")

# data structures
import pandas as pd
import numpy as np

# for graphs
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
%matplotlib inline

import plotly.express as px
import plotly.graph_objects as go
import plotly.offline as pyo

from plotly.subplots import make_subplots
from IPython.display import display

# Date & Time
import datetime as dt 

import warnings
warnings.filterwarnings("ignore")


In [2]:
# Get files

myfolder = Path("C:/Users/pdpc/Documents/MLfiles/2024 Berkeley AIML course/BerkAI Jupy & Codio Solutions/data/")


gdp = pd.read_csv(myfolder / "gdp.csv")

pop = pd.read_csv(myfolder / "population.csv")


In [5]:
gdp = gdp.drop(['Code'], axis=1, errors='ignore')

gdp = gdp.rename(columns={"GDP (constant 2010 US$)": "GDP"})

gdp.info()

pop = pop.drop(['Code'], axis=1, errors='ignore')

pop = pop.rename(columns={"Total population (Gapminder, HYDE & UN)": "Population"})

pop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8869 entries, 0 to 8868
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Entity  8869 non-null   object 
 1   Year    8869 non-null   int64  
 2   GDP     8869 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 208.0+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46883 entries, 0 to 46882
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Entity      46883 non-null  object
 1   Year        46883 non-null  int64 
 2   Population  46883 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.1+ MB


In [41]:
gdptest = gdp.query('Year > 2015')

gdptest.groupby(['Year']).nlargest(5, 'GDP')

AttributeError: 'DataFrameGroupBy' object has no attribute 'nlargest'

In [45]:
gdptest.groupby(['Year'])['GDP'].nlargest(2)

Year      
2016  8493    1.692033e+13
      1782    9.505157e+12
2017  8494    1.730498e+13
      1783    1.016101e+13
Name: GDP, dtype: float64

In [74]:
# Get Significant countries List

# I am defining significant countries as those which are Top 15 since 1990 in any year by GDP or Pop

#Step 0
# Number of Top countries
N = 10
# Since which year
yr = 1990

#Step 1
# Get Unique list of all countries which have been in Top 10 GDP in any year since 2000
A = gdp.query('Year > @yr').sort_values(['Year', 'GDP'], ascending=[True, False]).groupby(['Year']).head(N).Entity.unique()
A = A.tolist()

# Step 2
# Get Unique list of all countries which have been in Top 10 population in any year since 2000
B = pop.query('Year > @yr').sort_values(['Year', 'Population'], ascending=[True, False]).groupby(['Year']).head(N).Entity.unique()
B = B.tolist()

# Step 3
# Add both lists
C = list(set(A + B))

# Remove continents from the lists
D = ['World','Asia','Africa','Europe','Latin America','North America']
for i in D:
    try:
        C.remove(i)
    except ValueError:
        pass  # do nothing!
    
C.sort()

print("Number of countries that are Top" , N , "by GDP or population in any year since" , yr , ":" , len(C))

C


Number of countries that are Top 10 by GDP or population in any year since 1990 : 13


['Brazil',
 'Canada',
 'China',
 'France',
 'Germany',
 'India',
 'Indonesia',
 'Italy',
 'Japan',
 'Russia',
 'Spain',
 'United Kingdom',
 'United States']

In [52]:
# Get plot of significant countries since 2000

px.line(gdp.query('Entity in @C &  Year > 2000'), x = 'Year' , y = 'GDP' , color = 'Entity' , 
              color_discrete_sequence=px.colors.qualitative.Pastel)


In [53]:
# Join Tables

gdp_miniset = gdp.query('Entity in @C & Year > 2015 ' )

gdp_miniset

,Entity,Year,GDP
1782,China,2016,9.505157e+12
1783,China,2017,1.016101e+13
2918,France,2016,2.806036e+12
2919,France,2017,2.857089e+12
3129,Germany,2016,3.781699e+12
3130,Germany,2017,3.865759e+12
3788,India,2016,2.466177e+12
3789,India,2017,2.629542e+12
4260,Japan,2016,6.052672e+12
4261,Japan,2017,6.156329e+12


In [55]:
# Join Tables

pop_miniset = pop.query('Entity in @C &  Year > 2015 & Year < 2018')

pop_miniset

,Entity,Year,Population
8720,China,2016,1414049024
8721,China,2017,1421021952
14622,France,2016,64668000
14623,France,2017,64843000
15642,Germany,2016,82194000
15643,Germany,2017,82658000
18782,India,2016,1324516992
18783,India,2017,1338676992
20612,Japan,2016,127763000
20613,Japan,2017,127503000


In [59]:
# Join

joineddf = pd.merge(left = gdp_miniset , right = pop_miniset , how = 'left' , on = ['Entity' , 'Year'])

joineddf.sort_values(['Year' , 'GDP'], ascending=[True,False])

,Entity,Year,GDP,Population
10,United States,2016,1.692033e+13,323016000
0,China,2016,9.505157e+12,1414049024
8,Japan,2016,6.052672e+12,127763000
4,Germany,2016,3.781699e+12,82194000
2,France,2016,2.806036e+12,64668000
6,India,2016,2.466177e+12,1324516992
11,United States,2017,1.730498e+13,325084992
1,China,2017,1.016101e+13,1421021952
9,Japan,2017,6.156329e+12,127503000
5,Germany,2017,3.865759e+12,82658000


In [65]:
#

gdp_and_pop = pd.merge(gdp, pop, how='left', on=['Entity' , 'Year'])

gdp_and_pop['GDPperCapita'] = gdp_and_pop['GDP'] / gdp_and_pop['Population']

print(gdp.shape)
print(pop.shape)
print(gdp_and_pop.shape)

gdp_and_pop.info()

(8869, 3)
(46883, 3)
(8869, 5)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 8869 entries, 0 to 8868
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Entity        8869 non-null   object 
 1   Year          8869 non-null   int64  
 2   GDP           8869 non-null   float64
 3   Population    8851 non-null   float64
 4   GDPperCapita  8851 non-null   float64
dtypes: float64(3), int64(1), object(1)
memory usage: 415.7+ KB


In [78]:
px.line(gdp_and_pop.query('Entity in @C') , x='Year' , y='GDPperCapita', color='Entity')


In [80]:
gdp_and_pop.shape[0]

8869

In [87]:
px.scatter(gdp_and_pop.query('Entity in @C & Year == 2017'), x = "GDP", y = "GDPperCapita", color = "Entity", size ='Population')

In [114]:
#Life Expectancy Table

life_exp = pd.read_csv(myfolder / "life_expectancy.csv")

life_exp = life_exp.drop(['Code'], axis=1, errors='ignore')

life_exp.info()                 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18445 entries, 0 to 18444
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Entity           18445 non-null  object 
 1   Year             18445 non-null  int64  
 2   Life expectancy  18445 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 432.4+ KB


In [115]:
life_exp.head()

,Entity,Year,Life expectancy
0,Afghanistan,1950,27.638
1,Afghanistan,1951,27.878
2,Afghanistan,1952,28.361
3,Afghanistan,1953,28.852
4,Afghanistan,1954,29.350


In [124]:
# Merge life exp table to gdp_and_pop table

gdp_and_pop_lifeexp = pd.merge(gdp_and_pop, life_exp, how='left', on=['Entity' , 'Year'])

gdp_and_pop_lifeexp.head()

,Entity,Year,GDP,Population,GDPperCapita,Life expectancy
0,Afghanistan,2002,8.013233e+09,22601000.0,354.552149,56.784
1,Afghanistan,2003,8.689884e+09,23681000.0,366.955940,57.271
2,Afghanistan,2004,8.781610e+09,24727000.0,355.142564,57.772
3,Afghanistan,2005,9.762979e+09,25654000.0,380.563610,58.290
4,Afghanistan,2006,1.030523e+10,26433000.0,389.862222,58.826


In [112]:


fig = px.scatter(gdp_and_pop_lifeexp.query('Year==2017'), x = 'GDPperCapita', y = 'Life expectancy' , color = 'Entity')

fig.update_layout(    
    title = 'Life Expectancy by GDPperCapita', 
    xaxis_title = 'GDPperCapita', yaxis_title = 'LifeExp',
    width = 800,height = 600,
    template="simple_white")

fig.add_vline(x=gdp_and_pop_lifeexp['GDPperCapita'].median(), opacity = 1, line_width=2, line_color="green", 
              annotation_text="GDPpercapita median" , line_dash='dash')

fig.add_hline(y=gdp_and_pop_lifeexp['Life expectancy'].median(), opacity = 1, line_width=2, line_color="green",
              annotation_text="Life exp median" , line_dash='dash')

fig.add_vline(x=gdp_and_pop_lifeexp['GDPperCapita'].mean(), opacity = 1, line_width=2, line_color="red", 
              annotation_text="GDPpercapita mean" , line_dash='dash')

fig.add_hline(y=gdp_and_pop_lifeexp['Life expectancy'].mean(), opacity = 1, line_width=2, line_color="red",
              annotation_text="Life exp mean" , line_dash='dash')


fig.show()



In [113]:
px.scatter(gdp_and_pop_lifeexp.query('Year==2017'), x = 'GDPperCapita', y = 'Life expectancy' , color = 'Entity' , log_x = True)

In [121]:
# Continents

continents = pd.read_csv(myfolder / "continents.csv")

continents.head()

,Country,Continent
0,Algeria,Africa
1,Angola,Africa
2,Benin,Africa
3,Botswana,Africa
4,Burkina Faso,Africa


In [125]:
gdp_and_pop_lifeexp = pd.merge(gdp_and_pop_lifeexp, continents, how='left', left_on=['Entity'] , right_on=['Country'])

gdp_and_pop_lifeexp.head()

,Entity,Year,GDP,Population,GDPperCapita,Life expectancy,Country,Continent
0,Afghanistan,2002,8.013233e+09,22601000.0,354.552149,56.784,Afghanistan,South Asia
1,Afghanistan,2003,8.689884e+09,23681000.0,366.955940,57.271,Afghanistan,South Asia
2,Afghanistan,2004,8.781610e+09,24727000.0,355.142564,57.772,Afghanistan,South Asia
3,Afghanistan,2005,9.762979e+09,25654000.0,380.563610,58.290,Afghanistan,South Asia
4,Afghanistan,2006,1.030523e+10,26433000.0,389.862222,58.826,Afghanistan,South Asia


In [126]:
px.box(gdp_and_pop_lifeexp.query('Year==2017').dropna(), y = 'Life expectancy' , color = 'Continent')

In [127]:
px.violin(gdp_and_pop_lifeexp.query('Year==2017').dropna(), y = 'Life expectancy' , color = 'Continent')

In [130]:
# Joint Plots

px.scatter(gdp_and_pop_lifeexp.query('Year==2017'), x = 'GDPperCapita', y = 'Life expectancy',
                marginal_x='histogram' , marginal_y='histogram')
